# Primer Proyecto Individual en la Etapa de Labs en HENRY.

#### Sección para la limpieza de los datos.

##### Consideraciones a tratar en este paso:

* Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos y unirlos al dataset de nuevo para hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos..

* Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

* Los valores nulos del campo release date deben eliminarse.

* De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

* Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

* Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [3]:
import pandas as pd
import os

# Limpieza del dataset "movies.csv"

In [4]:
# Extraemos el dataset hacia un dataframe en python.
movies_df = pd.read_csv('Data/movies_dataset.csv')

# Seleccionamos las columnas que necesitamos y eliminamos las que nos piden: video,imdb_id,adult,original_title,poster_path,homepage. Y renombramos algunas.
movies_df = movies_df[['id', 'title', 'budget', 'original_language', 'release_date', 'revenue', 'runtime', 'production_companies',  
        'belongs_to_collection', 'production_countries', 'genres', 'overview', 'popularity',
        'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count']]
movies_df.columns = ['id_peli', 'title', 'budget', 'original_language', 'release_date', 'revenue', 'runtime', 'production_companies',  
        'belongs_to_collection', 'production_countries', 'genres', 'overview', 'popularity',
        'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count']

# Eliminamos los campos nulos de la columna "release_date".
movies_df = movies_df.dropna(subset=['release_date']).reset_index(drop=True)

# Eliminamos películas duplicadas. Para este caso, tomamos las películas duplicadas con fecha de lanzamiento más reciente.
df = movies_df[movies_df.title.isin(list(movies_df[movies_df.title.duplicated()].title))]
df['row_count'] = df.sort_values(by=['title','release_date'],ascending=False).groupby(['title']).cumcount()+1
index = list(df[df.row_count > 1].id_peli)
movies_df = movies_df.set_index('id_peli')
movies_df = movies_df.drop(index).reset_index()

# Transformando los datos del "id" de película a enteros: Me di cuenta mediante este proceso de que existen 3 "id's" que son fechas, por lo que eliminé dichas filas.
index = []
for i in range(len(movies_df)):
    try:
        movies_df.loc[i,['id_peli']] = int(movies_df.id_peli[i])
    except ValueError:
        index.append(i)

# Visualizamos las filas que no contienen datos relevantes: index = [19714,29472,35543]      
print('Filas eliminadas del dataset por no contener información relevante: ',movies_df.loc[index])

# Eliminamos las filas.
movies_df = movies_df.drop(index)

# Seleccionamos las películas con "status" = 'Released'.
movies_df = movies_df[movies_df.status == 'Released']

# ====================================================================================================================================

# Por último vamos a examinar si existen filas duplicadas en nuestro dataframe.
# Utilizamos primeramente el método drop_duplicates().
movies_df = movies_df.drop_duplicates()

# Después buscamos si existen "id's" repetidos después del método.
dfx = movies_df.groupby('id_peli')['id_peli'].count() > 1
dfx = pd.DataFrame(dfx)
index = list(dfx[dfx.id_peli == True].index)

# Creamos un DF temporal con el objetivo de obtener las filas con 'id's' duplicados.
dfx = pd.DataFrame()
for i in index:
    dfx = pd.concat([dfx,movies_df[movies_df.id_peli == i]])

# Tomamos las filas duplicadas que están en segundo lugar y las eliminamos.
index = list(dfx[1::2].index)
movies_df = movies_df.drop(index).reset_index(drop=True)

# ====================================================================================================================================

# Rellenar valores nulos en los campos "budget" y "revenue" por 0.
movies_df['revenue'] = movies_df.revenue.fillna(0)
movies_df['budget'] = movies_df.budget.fillna(0)

# ====================================================================================================================================

# De haber fechas, deberán tener el formato AAAA-mm-dd, además, deberán crear la columna "release_year", donde extraerán el año de la fecha de estreno.
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])
movies_df['release_year'] = movies_df['release_date'].apply(lambda x: x.year)

# ====================================================================================================================================

# Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.
movies_df['budget'] = movies_df.budget.astype(float)
def returns(x,y):
    movies_df['return'] = pd.Series()
    for i in range(len(x)):
        movies_df['return'][i] = x[i]/y[i] if y[i] != 0 else 0
returns(movies_df.revenue,movies_df.budget)

# ====================================================================================================================================

# Creamos una función lambda que nos va a ayudar a pasar los strings a listas de diccionarios.
a_lista = lambda x: eval(x)

# Separamos las columnas con sus respectivos "id_peli" en 5 diferentes DF's.
df1 = movies_df[['id_peli','production_companies']].reset_index(drop=True)
df2 = movies_df[['id_peli','belongs_to_collection']].reset_index(drop=True)
df3 = movies_df[['id_peli','production_countries']].reset_index(drop=True)
df4 = movies_df[['id_peli','genres']].reset_index(drop=True)
df5 = movies_df[['id_peli','spoken_languages']].reset_index(drop=True)

# Ahora que ya hemos utilizado las columnas que necesitamos para desanidar las columnas anidadas, podemos eliminarlas de nuestro dataframe de películas, ya que la información está en
# nuevos dataframes.
movies_df = movies_df[['id_peli', 'title', 'budget', 'revenue', 'return', 'original_language', 'release_year', 'release_date', 'runtime', 'popularity', 'vote_average', 'vote_count']]

# ====================================================================================================================================

# Convertimos los campos anidados de nuestro dataset inicial, de strings a listas de diccionarios. En total son 5 columnas anidadas.
df1['production_companies'] = df1['production_companies'].apply(a_lista)

df2 = df2.dropna().reset_index(drop=True)
df2['belongs_to_collection'] = df2['belongs_to_collection'].apply(a_lista)

df3['production_countries'] = df3['production_countries'].apply(a_lista)

df4['genres'] = df4['genres'].apply(a_lista)

df5 = df5.dropna().reset_index(drop=True)
df5['spoken_languages'] = df5['spoken_languages'].apply(a_lista)

# ====================================================================================================================================

# Dejamos la columna "release_date" como datetime. Y capitalizamos las letras de los titulos para homogenizar.
movies_df['release_date'] = pd.to_datetime(movies_df.release_date)
movies_df['title'] = movies_df.title.apply(lambda x: x.title())
movies_df['original_language'] = movies_df.original_language.astype(str)
movies_df['original_language'] = movies_df.original_language.apply(lambda x: x.lower())


if not os.path.exists('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/'):
    os.mkdir('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/')
movies_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/movies.csv',index=False)

# Visualizamos el dataframe.
pd.read_csv('clean_data/movies.csv')

C:\Users\tinma\AppData\Local\Temp\ipykernel_25160\3219681389.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('Data/movies_dataset.csv')
C:\Users\tinma\AppData\Local\Temp\ipykernel_25160\3219681389.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_count'] = df.sort_values(by=['title','release_date'],ascending=False).groupby(['title']).cumcount()+1


Filas eliminadas del dataset por no contener información relevante:            id_peli title                            budget original_language  \
17925  1997-08-20   NaN  /ff9qCepilowshEtG2GYWwzt2bs4.jpg             104.0   
27013  2012-09-29   NaN  /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg              68.0   
32768  2014-01-01   NaN  /zaSf5OG7V8X8gqFvly88zDdRm46.jpg              82.0   

      release_date  revenue  runtime production_companies  \
17925            1      NaN      NaN                False   
27013           12      NaN      NaN                False   
32768           22      NaN      NaN                False   

      belongs_to_collection production_countries  \
17925              0.065736                  6.0   
27013              1.931659                  7.0   
32768              2.185485                  4.3   

                                                  genres  overview  \
17925  [{'name': 'Carousel Productions', 'id': 11176}...  Released   
27013  [{'name': 'An

C:\Users\tinma\AppData\Local\Temp\ipykernel_25160\3219681389.py:76: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  movies_df['return'] = pd.Series()
C:\Users\tinma\AppData\Local\Temp\ipykernel_25160\3219681389.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['return'][i] = x[i]/y[i] if y[i] != 0 else 0


,id_peli,title,budget,revenue,return,original_language,release_year,release_date,runtime,popularity,vote_average,vote_count
0,862,Toy Story,30000000.0,373554033.0,12.451801,en,1995,1995-10-30,81.0,21.946943,7.7,5415.0
1,8844,Jumanji,65000000.0,262797249.0,4.043035,en,1995,1995-12-15,104.0,17.015539,6.9,2413.0
2,15602,Grumpier Old Men,0.0,0.0,0.000000,en,1995,1995-12-22,101.0,11.712900,6.5,92.0
3,31357,Waiting To Exhale,16000000.0,81452156.0,5.090760,en,1995,1995-12-22,127.0,3.859495,6.1,34.0
4,11862,Father Of The Bride Part Ii,0.0,76578911.0,0.000000,en,1995,1995-02-10,106.0,8.387519,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...
41760,222848,Caged Heat 3000,0.0,0.0,0.000000,en,1995,1995-01-01,85.0,0.661558,3.5,1.0
41761,111109,Century Of Birthing,0.0,0.0,0.000000,tl,2011,2011-11-17,360.0,0.178241,9.0,3.0
41762,67758,Betrayal,0.0,0.0,0.000000,en,2003,2003-08-01,90.0,0.903007,3.8,6.0
41763,227506,Satan Triumphant,0.0,0.0,0.000000,en,1917,1917-10-21,87.0,0.003503,0.0,0.0


In [5]:
df3

,id_peli,production_countries
0,862,"[{'iso_3166_1': 'US', 'name': 'United States o..."
1,8844,"[{'iso_3166_1': 'US', 'name': 'United States o..."
2,15602,"[{'iso_3166_1': 'US', 'name': 'United States o..."
3,31357,"[{'iso_3166_1': 'US', 'name': 'United States o..."
4,11862,"[{'iso_3166_1': 'US', 'name': 'United States o..."
...,...,...
41760,222848,"[{'iso_3166_1': 'US', 'name': 'United States o..."
41761,111109,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]"
41762,67758,"[{'iso_3166_1': 'US', 'name': 'United States o..."
41763,227506,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]"


### Desanidado de columnas y creación de 5 nuevos datasets

In [25]:
# ====================================================================================================================================
# Creación del DF "productoras_df" y del DF "id_productoras_df".
# Lo que sigue es obtener el dataframe de las productoras asociadas a cada película.
# sourcery skip: use-fstring-for-concatenation
dfx = pd.DataFrame()
for i in range(len(df1)):
    df = pd.DataFrame(df1.production_companies[i])
    df['id'] = df1.id_peli[i]
    dfx = pd.concat([dfx,df])
dfx = dfx.reset_index(drop=True)
dfx.columns = ['productora','id_peli']

# Ahora creamos el dataframe de las productoras únicas con un id único.
productoras_df = dfx.productora.unique()
productoras_df = pd.DataFrame(productoras_df,columns=['productora'])
productoras_df = productoras_df.reset_index()
productoras_df.columns = ['id_prod','productora']
productoras_df['id_prod'] = productoras_df.id_prod.apply(lambda x: x+1)
productoras_df['productora'] = productoras_df.productora.apply(lambda x: x.title())


# Por último agregamos los "id's" de las productoras al dataframe de las productoras asociadas a películas y eliminamos la columna que contiene el nombre de la productora para reducir el 
# uso de memoria.
dfx = dfx.merge(productoras_df)
dfx = dfx.drop('productora',axis=1)
id_productoras_df = dfx
id_productoras_df['id_peli'] = id_productoras_df.id_peli.astype(int)

# Creamos los .csv llamados "productoras", "prod_movies".
productoras_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/productoras.csv',index=False)
id_productoras_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/prod_movies.csv',index=False)

# ====================================================================================================================================
# Creación del DF "franquicias_df" y del DF "id_franquicias_df"
# Obtenemos el dataframe de las franquicias asociadas a cada película.
dfx = pd.DataFrame()
for i in range(len(df2)):
    df = pd.DataFrame([df2.belongs_to_collection[i]])
    df['id'] = df2.id_peli[i]
    dfx = pd.concat([dfx,df])
dfx = dfx.reset_index(drop=True)

# Ahora creamos el dataframe de las franquicias únicas con un id único.
dfx = dfx.drop(['poster_path','backdrop_path'],axis=1)
dfx.columns = ['id_peli','franquicia']
franquicias_df = dfx.franquicia.unique()
franquicias_df = pd.DataFrame(franquicias_df,columns=['franquicia']).reset_index()
franquicias_df.columns = ['id_franq','franquicia']
franquicias_df['id_franq'] = franquicias_df.id_franq.apply(lambda x: x+1)
franquicias_df = franquicias_df[['id_franq','franquicia']]
franquicias_df['franquicia'] = franquicias_df.franquicia.apply(lambda x: x.title())

# Por último, agregamos los "id's" de las franquicias al dataframe de las franquicias asociadas a películas y eliminamos la columna que contiene el nombre de la franquicia para reducir el 
# uso de memoria.
id_franquicias_df = dfx.merge(franquicias_df)
id_franquicias_df = id_franquicias_df.drop(['franquicia'],axis=1)
id_franquicias_df['id_peli'] = id_franquicias_df.id_peli.astype(int) 

# Creamos los .csv llamados "productoras", "prod_movies".
franquicias_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/franquicias.csv',index=False)
id_franquicias_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/franq_movies.csv',index=False)

# ====================================================================================================================================
# Creación del DF "paises_df" y del DF "id_paises_df"
# Obtenemos el dataframe de los países asociados a cada película.
dfx = pd.DataFrame()
for i in range(len(df3)):
    df = pd.DataFrame(df3.production_countries[i])
    df['id'] = df3.id_peli[i]
    dfx = pd.concat([dfx,df])
dfx = dfx.reset_index(drop=True)

# Ahora creamos el dataframe de los países únicos con un id único.
dfx.columns = ['iso_3166_1','pais','id_peli']
dfx['dummy'] = dfx.iso_3166_1 + ' ' + dfx.pais
paises_df = dfx.dummy.unique()
paises_df = pd.DataFrame(paises_df,columns=['pais']).reset_index()
paises_df.columns = ['id_pais','pais']
paises_df['abrev'] = paises_df.pais.apply(lambda x: x[:2])
paises_df['pais'] = paises_df.pais.apply(lambda x: x[3:])
paises_df['id_pais'] = paises_df.id_pais.apply(lambda x: x + 1)
paises_df['pais'] = paises_df.pais.apply(lambda x: x.title())

# Por último, agregamos los "id's" de los países al dataframe de los países asociadas a películas y eliminamos la columna que contiene el nombre del país para reducir el 
# uso de memoria.
dfx['pais'] = dfx.pais.apply(lambda x: x.title())
id_paises_df = dfx.merge(paises_df)
id_paises_df = id_paises_df.drop(['iso_3166_1','pais','dummy','abrev'],axis=1)
id_paises_df['id_peli'] = id_paises_df.id_peli.astype(int)

# Creamos los .csv llamados "productoras", "prod_movies".
paises_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/paises.csv',index=False)
id_paises_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/pais_movies.csv',index=False)

# ====================================================================================================================================
# Creación del DF "generos_df" y del DF "id_generos_df"
# Obtenemos el dataframe de los generos asociados a cada película.
dfx = pd.DataFrame()
for i in range(len(df4)):
    df = pd.DataFrame(df4.genres[i])
    df['id'] = df4.id_peli[i]
    dfx = pd.concat([dfx,df])
dfx = dfx.reset_index(drop=True)

# Ahora creamos el dataframe de los generos únicos con un id único.
dfx.columns = ['id_peli','genero']
generos_df = pd.DataFrame(dfx.genero.unique(),columns=['genero']).reset_index()
generos_df.columns = ['id_gen','genero']
generos_df['id_gen'] = generos_df.id_gen.apply(lambda x: x + 1)
generos_df['genero'] = generos_df.genero.apply(lambda x: x.title())

# Por último, agregamos los "id's" de los generos al dataframe de los generos asociadas a películas y eliminamos la columna que contiene el nombre del genero para reducir el 
# uso de memoria.
id_generos_df = dfx.merge(generos_df)
id_generos_df = id_generos_df.drop(['genero'],axis=1)
id_generos_df['id_peli'] = id_generos_df.id_peli.astype(int)
id_generos_df.columns = ['id_peli','id_gen']

# Creamos los .csv llamados "productoras", "prod_movies".
generos_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/generos.csv',index=False)
id_generos_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/gen_movies.csv',index=False)

# ====================================================================================================================================
# Creación del DF "lenguajes_df" y del DF "id_lenguajes_df"
# Obtenemos el dataframe de los lenguajes asociados a cada película.
dfx = pd.DataFrame()
for i in range(len(df5)):
    df = pd.DataFrame(df5.spoken_languages[i])
    df['id'] = df5.id_peli[i]
    dfx = pd.concat([dfx,df])
dfx = dfx.reset_index(drop=True)

# Ahora creamos el dataframe de los lenguajes únicos con un id único.
dfx.columns = ['iso_639_1','lenguaje','id_peli']
dfx['dummy'] = dfx.iso_639_1 + ' ' + dfx.lenguaje
lenguajes_df = pd.DataFrame(dfx.dummy.unique(),columns=['lenguaje'])
lenguajes_df['abrev'] = lenguajes_df.lenguaje.apply(lambda x: x[:2])
lenguajes_df['lenguaje'] = lenguajes_df.lenguaje.apply(lambda x: x[3:])
lenguajes_df = lenguajes_df.reset_index()
lenguajes_df.columns = ['id_leng','lenguaje','abrev']
lenguajes_df['id_leng'] = lenguajes_df.id_leng.apply(lambda x: x + 1)
lenguajes_df = lenguajes_df[lenguajes_df.abrev.isin(list(movies_df.original_language.unique()))]
lenguajes_df['lenguaje'] = lenguajes_df.lenguaje.apply(lambda x: x.title())
lenguajes_df['abrev'] = lenguajes_df.abrev.apply(lambda x: x.lower())

# Por último, agregamos los "id's" de los lenguajes al dataframe de los lenguajes asociadas a películas y eliminamos la columna que contiene el nombre del lenguaje para reducir el 
# uso de memoria.
id_lenguajes_df = dfx.merge(lenguajes_df)
id_lenguajes_df = id_lenguajes_df.drop(['iso_639_1','lenguaje','dummy','abrev'],axis=1)
id_lenguajes_df['id_peli'] = id_lenguajes_df.id_peli.astype(int)

# Creamos los .csv llamados "productoras", "prod_movies".
lenguajes_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/lenguajes.csv',index=False)
id_lenguajes_df.to_csv('C:/Users/tinma/OneDrive/Escritorio/HENRY/PI1_MLOps_HENRY/clean_data/leng_movies.csv',index=False)

In [22]:
paises_df

,id_pais,pais,abrev
0,1,United States Of America,US
1,2,Germany,DE
2,3,United Kingdom,GB
3,4,France,FR
4,5,Italy,IT
...,...,...,...
155,156,Antarctica,AQ
156,157,Gibraltar,GI
157,158,Brunei Darussalam,BN
158,159,Honduras,HN


In [26]:
id_paises_df

,id_peli,id_pais
0,862,1
1,8844,1
2,15602,1
3,31357,1
4,11862,1
...,...,...
45552,132873,156
45553,16403,157
45554,277690,158
45555,321530,159


# Limpieza del dataset "credits.csv"

In [25]:
# ====================================================================================================================================
# Ingresamos el dataset.
credits_df = pd.read_csv('Data/credits.csv')

# Dividimos el dataset en dos dataframe's
elenco_df = credits_df[['id','cast']]
productores_df = credits_df[['id','crew']]

# ====================================================================================================================================
# Eliminamos los nan, reestablecemos el índice y convertimos las strings en listas de diccionarios de ambos datasets.
productores_df = productores_df.dropna().reset_index(drop=True)
productores_df['crew'] = productores_df['crew'].apply(a_lista)
elenco_df = elenco_df.dropna().reset_index(drop=True)
elenco_df['cast'] = elenco_df['cast'].apply(a_lista)

# ====================================================================================================================================
# Desanidamos la columna "crew".
dfx = pd.DataFrame()
for i in range(len(productores_df)):
    df = pd.DataFrame(productores_df.crew[i])
    if not df.empty:
        df = df[['id', 'job', 'name']][df.job == 'Director']
        df = df.drop('job',axis=1)
        df['id'] = productores_df.id[i]
        dfx = pd.concat([dfx,df])
dfx = dfx.reset_index(drop=True)

# Obtenemos solo los directores del dataset "credits.csv" y la tabla que asocia películas con directores.
id_directores_df = dfx
id_directores_df.columns = ['id_peli', 'director']
directores_df = id_directores_df.director.unique()
directores_df = pd.DataFrame(directores_df).reset_index()
directores_df.columns = ['id_dir','director']
directores_df['id_dir'] = directores_df.id_dir.apply(lambda x: x + 1)
directores_df['director'] = directores_df.director.apply(lambda x: x.title())
id_directores_df = id_directores_df.merge(directores_df).drop('director',axis=1)

# ====================================================================================================================================
# Desanidamos la columna "cast" y creamos el dataframe de personajes.
dfx = pd.DataFrame()
for i in range(len(elenco_df)):
    df = pd.DataFrame(elenco_df.cast[i])
    if not df.empty:
        df = df[['id', 'name']]
        df['id'] = elenco_df.id[i]
        dfx = pd.concat([dfx,df])
dfx = dfx.reset_index(drop=True)
dfx.columns = ['id_peli', 'actor']
id_personajes_actores_df = dfx

# También creamos el dataset de actores con el mismo objetivo.
dfx = id_personajes_actores_df.actor.unique()
dfx = pd.DataFrame(dfx).reset_index()
dfx.columns = ['id_act','actor']
dfx['id_act'] = dfx.id_act.apply(lambda x: x + 1)
dfx['actor'] = dfx.actor.apply(lambda x: x.title())
actores_df = dfx

# Ahora eliminamos la columna de actores y de personajes para solo dejar "id's"
id_personajes_actores_df = id_personajes_actores_df.merge(actores_df)
id_personajes_actores_df = id_personajes_actores_df.drop(['actor'],axis=1)
id_personajes_actores_df

# ====================================================================================================================================
# Creamos los archivos .csv de los dataframe's resultantes.
directores_df.to_csv('clean_data/directores.csv',index=False)
id_directores_df.to_csv('clean_data/dir_movies.csv',index=False)
actores_df.to_csv('clean_data/actores.csv',index=False)
id_personajes_actores_df.to_csv('clean_data/pers_movies.csv',index=False)

# Sección de desarrollo del dataset para el modelo de recomendación

Esta parte de la limpieza se desarrolló a la par del desarrollo del EDA.

In [81]:
# franquicia, genero, productora, paises, lenguaje, overview, release_year
df = pd.read_csv('clean_data/movies.csv')
dfx = pd.read_csv('Data/movies_dataset.csv')
df = df[['id_peli', 'title', 'original_language', 'popularity', 'vote_average', 'vote_count', 'release_year']]
df

C:\Users\tinma\AppData\Local\Temp\ipykernel_24196\196826973.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfx = pd.read_csv('Data/movies_dataset.csv')


,id_peli,title,original_language,popularity,vote_average,vote_count,release_year
0,862,Toy Story,en,21.946943,7.7,5415.0,1995
1,8844,Jumanji,en,17.015539,6.9,2413.0,1995
2,15602,Grumpier Old Men,en,11.712900,6.5,92.0,1995
3,31357,Waiting To Exhale,en,3.859495,6.1,34.0,1995
4,11862,Father Of The Bride Part Ii,en,8.387519,5.7,173.0,1995
...,...,...,...,...,...,...,...
41760,222848,Caged Heat 3000,en,0.661558,3.5,1.0,1995
41761,111109,Century Of Birthing,tl,0.178241,9.0,3.0,2011
41762,67758,Betrayal,en,0.903007,3.8,6.0,2003
41763,227506,Satan Triumphant,en,0.003503,0.0,0.0,1917


In [82]:
dfx = dfx[['id','overview']]
index = []
for i in range(len(dfx)):
    try:
        dfx.loc[i,['id']] = int(dfx.id[i])
    except ValueError:
        index.append(i)
dfx.columns = ['id_peli','overview']
dfx = dfx.drop(index)
dfx

,id_peli,overview
0,862,"Led by Woody, Andy's toys live happily in his ..."
1,8844,When siblings Judy and Peter discover an encha...
2,15602,A family wedding reignites the ancient feud be...
3,31357,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Just when George Banks has recovered from his ...
...,...,...
45461,439050,Rising and falling between a man and woman.
45462,111109,An artist struggles to finish his work while a...
45463,67758,"When one of her hits goes wrong, a professiona..."
45464,227506,"In a small town live two brothers, one a minis..."


### Limpieza que surgió durante el desarrollo del modelo.

Parte del código que vamos a utilizar para generar esta parte de la limpieza la podemos obtener del EDA. Aunque también en esta parte comienza una parte del preprocesamiento

In [181]:
franq = pd.read_csv('clean_data/franquicias.csv')
id_franq = pd.read_csv('clean_data/franq_movies.csv')
franq = id_franq.merge(franq).drop('id_franq',axis=1)
df = pd.read_csv('clean_data/movies.csv')
df = df.merge(franq,how='left')
lista = list(df[['title','vote_count','popularity','release_year']].sort_values(by=['vote_count','popularity','release_year'])[:len(df) - 36000].index)
df = df.drop(lista)
lista = list(df[['title','vote_count','popularity','release_year']][df.release_year < 1990].sort_values(by=['release_year']).index)
df = df.drop(lista)
lista = list(df[['id_peli','title','vote_count', 'vote_average','popularity','release_year']].sort_values(by=['vote_count','popularity','release_year'])[df.vote_count >= 15].id_peli)
df = df[df.id_peli.apply(lambda x: x in lista)].reset_index(drop=True)
df

C:\Users\tinma\AppData\Local\Temp\ipykernel_25852\1153106459.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = list(df[['id_peli','title','vote_count', 'vote_average','popularity','release_year']].sort_values(by=['vote_count','popularity','release_year'])[df.vote_count >= 15].id_peli)


,id_peli,title,budget,revenue,return,original_language,release_year,release_date,runtime,popularity,vote_average,vote_count,franquicia
0,862,Toy Story,30000000.0,373554033.0,12.451801,en,1995,1995-10-30,81.0,21.946943,7.7,5415.0,Toy Story Collection
1,8844,Jumanji,65000000.0,262797249.0,4.043035,en,1995,1995-12-15,104.0,17.015539,6.9,2413.0,NaN
2,15602,Grumpier Old Men,0.0,0.0,0.000000,en,1995,1995-12-22,101.0,11.712900,6.5,92.0,Grumpy Old Men Collection
3,31357,Waiting To Exhale,16000000.0,81452156.0,5.090760,en,1995,1995-12-22,127.0,3.859495,6.1,34.0,NaN
4,11862,Father Of The Bride Part Ii,0.0,76578911.0,0.000000,en,1995,1995-02-10,106.0,8.387519,5.7,173.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13131,432789,The Incredible Jessica James,0.0,0.0,0.000000,en,2017,2017-01-27,83.0,5.667067,6.2,37.0,NaN
13132,434873,It Stains The Sands Red,0.0,0.0,0.000000,en,2016,2016-10-08,92.0,8.471866,5.6,21.0,NaN
13133,18098,Arabian Nights,0.0,0.0,0.000000,en,2000,2000-04-30,175.0,2.266456,6.9,21.0,NaN
13134,455661,In A Heartbeat,0.0,0.0,0.000000,en,2017,2017-06-01,4.0,20.821780,8.3,146.0,NaN


In [159]:
df[['title','vote_count','popularity','release_year']].sort_values(by='release_year')[:50]

KeyError: "None of [Index(['title', 'vote_count', 'popularity', 'release_year'], dtype='object')] are in the [columns]"

### Más variaciones a la limpieza y preprocesamiento

Ahora vamos a utilizar el acercamiento en donde utilizamos el género, la productora, la franquicia, el titulo y el original_language. Nos dimos cuenta que actores, productoras y directores nos daban recomendaciones poco acertadas.

In [150]:
franq = pd.read_csv('clean_data/franquicias.csv')
id_franq = pd.read_csv('clean_data/franq_movies.csv')
franq = id_franq.merge(franq).drop('id_franq',axis=1)
df = pd.read_csv('clean_data/movies.csv')
df = df.merge(franq,how='left')
lista = list(df[['title','vote_count','popularity','release_year']].sort_values(by=['vote_count','popularity','release_year'])[:len(df) - 36000].index)
df = df.drop(lista)
df = df[['title','vote_count','popularity','release_year']][df.release_year > 1990].sort_values(by=['release_year'])

,title,vote_count,popularity,release_year
1011,The Double Life Of Veronique,154.0,9.661817,1991
29122,Lee Rock,3.0,1.807175,1991
29183,Robotrix,3.0,0.667275,1991
29265,Ultrà,10.0,0.266692,1991
29382,The Crucifer Of Blood,5.0,0.512575,1991
...,...,...,...,...
40957,Dragonheart: Battle For The Heartfire,25.0,7.794894,2017
40956,Drone,55.0,8.413334,2017
40099,Asylum Of Darkness,4.0,0.474061,2017
40995,What Happened To Monday,598.0,60.581223,2017


In [248]:
franq = pd.read_csv('clean_data/franquicias.csv')
id_franq = pd.read_csv('clean_data/franq_movies.csv')
franq = id_franq.merge(franq).drop('id_franq',axis=1)
df = pd.read_csv('clean_data/movies.csv')
df = df.merge(franq,how='left')
lista = list(df[['title','vote_count','popularity','release_year']].sort_values(by=['vote_count','popularity','release_year'])[:len(df) - 36000].index)
df = df.drop(lista)
lista = list(df[['title','vote_count','popularity','release_year']][df.release_year < 1975].sort_values(by=['release_year']).index)
df = df.drop(lista)
lista = list(df[['id_peli','title','vote_count', 'vote_average','popularity','release_year']].sort_values(by=['vote_count','popularity','release_year'])[df.vote_count >= 15].id_peli)
df = df[df.id_peli.apply(lambda x: x in lista)].reset_index(drop=True)
df = df[['id_peli',	'title', 'franquicia']]

genero = pd.read_csv('clean_data/generos.csv')
id_genero = pd.read_csv('clean_data/gen_movies.csv')
productora = pd.read_csv('clean_data/productoras.csv')
id_productora = pd.read_csv('clean_data/prod_movies.csv')
actor = pd.read_csv('clean_data/actores.csv')
id_actor = pd.read_csv('clean_data/pers_movies.csv')
director = pd.read_csv('clean_data/directores.csv')
id_director = pd.read_csv('clean_data/dir_movies.csv')

genero = id_genero.merge(genero).drop('id_gen',axis=1)
productora = id_productora.merge(productora).drop('id_prod',axis=1)
actor = id_actor.merge(actor).drop('id_act',axis=1)
director = id_director.merge(director).drop('id_dir',axis=1)

C:\Users\tinma\AppData\Local\Temp\ipykernel_25852\3349443906.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = list(df[['id_peli','title','vote_count', 'vote_average','popularity','release_year']].sort_values(by=['vote_count','popularity','release_year'])[df.vote_count >= 15].id_peli)


In [238]:
df

,id_peli,title,franquicia
0,862,Toy Story,Toy Story Collection
1,8844,Jumanji,NaN
2,15602,Grumpier Old Men,Grumpy Old Men Collection
3,31357,Waiting To Exhale,NaN
4,11862,Father Of The Bride Part Ii,NaN
...,...,...,...
15071,432789,The Incredible Jessica James,NaN
15072,434873,It Stains The Sands Red,NaN
15073,18098,Arabian Nights,NaN
15074,455661,In A Heartbeat,NaN


In [242]:
act_list = []
for x in actor.id_peli.unique():
    lista_1 = [x]
    lista_1.extend(
        actor[actor.id_peli == x].loc[i].actor.replace(' ', '')
        for i in actor[actor.id_peli == x].index
    )
    act_list.append(lista_1)
    
dir_list = []
for x in director.id_peli.unique():
    lista_1 = [x]
    lista_1.extend(
        director[director.id_peli == x].loc[i].director.replace(' ', '')
        for i in director[director.id_peli == x].index
    )
    dir_list.append(lista_1)
    
gen_list = []
for x in genero.id_peli.unique():
    lista_1 = [x]
    lista_1.extend(
        genero[genero.id_peli == x].loc[i].genero.strip()
        for i in genero[genero .id_peli == x].index
    )
    gen_list.append(lista_1)
    
    

lista = ['id_peli']
lista.extend(f'dir_{i}' for i in range(41))
dir_df = pd.DataFrame(dir_list,columns=lista).set_index('id_peli')

lista = ['id_peli']
lista.extend(f'gen_{i}' for i in range(8))
gen_df = pd.DataFrame(gen_list,columns=lista).set_index('id_peli')

lista = ['id_peli']
lista.extend(f'act_{i}' for i in range(306))
act_df = pd.DataFrame(act_list,columns=lista).set_index('id_peli')

# # act_df = act_df.iloc[:,:50]

In [227]:
act_df

,act_0,act_1,act_2,act_3,act_4,act_5,act_6,act_7,act_8,act_9,...,act_40,act_41,act_42,act_43,act_44,act_45,act_46,act_47,act_48,act_49
id_peli,,,,,,,,,,,,,,,,,,,,,
862,TomHanks,TimAllen,DonRickles,JimVarney,WallaceShawn,JohnRatzenberger,AnniePotts,JohnMorris,LaurieMetcalf,R.LeeErmey,...,None,None,None,None,None,None,None,None,None,None
568,TomHanks,XanderBerkeley,GabrielJarret,KevinBacon,EdHarris,EndreHules,BillPaxton,GarySinise,KathleenQuinlan,MaryKateSchellhardt,...,None,None,None,None,None,None,None,None,None,None
13,TomHanks,MykeltiWilliamson,AaronMichaelLacey,StephenBridgewater,SallyField,RobinWright,GarySinise,JoeAlaskey,MaryEllenTrainor,SiobhanFallon,...,JohnWilliamGalt,IsabelRose,RichardD'Alessandro,GeoffreyBlake,VanessaRoth,DickCavett,TiffanySalerno,TiffanySalerno,LazarusJackson,LazarusJackson
9800,TomHanks,AnnaDeavereSmith,MarySteenburgen,AntonioBanderas,JoeyPerillo,CharlesTechman,LisaTalerico,TomDetrik,RobertaMaxwell,GeneBorkan,...,BuzzKilman,MarkSorensenJr.,JeffreyWilliamson,StephanieRothHaberle,BillRowe,FordWheeler,JuliusErving,GaryGoetzman,KennethUtt,JimRoche
858,TomHanks,DanaIvey,RossMalinger,DavidHydePierce,RosieO'Donnell,GabyHoffmann,RitaWilson,MegRyan,BillPullman,FrancesConroy,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264269,MariyaFomina,DmitriYendaltsev,BorisPolunin,ViktoriyaRaykova,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
370213,SonitaAlidazeh,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
75608,RobertGardner,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [244]:
# prod_df['productoras'] = pd.Series()
# for i in range(len(prod_df.columns)):
#     prod_df.iloc[:,i] = prod_df.iloc[:,i].astype(str)
# for x in prod_df.index:
#     lista_x = list(prod_df.loc[x,:].unique())
#     try:
#         lista_x.remove('nan')
#         lista_x.remove('None')
#     except ValueError:
#         pass
#     prod_df.loc[x,'productoras'] = ' '.join(lista_x)
# for i in prod_df.columns:
#     if i != 'productoras':
#         prod_df.drop(i,axis=1,inplace=True)
        
        
    
act_df['elenco'] = pd.Series()
for i in range(len(act_df.columns)):
    act_df.iloc[:,i] = act_df.iloc[:,i].astype(str)
for x in act_df.index:
    lista_x = list(act_df.loc[x,:].unique())
    try:
        lista_x.remove('nan')
        lista_x.remove('None')
    except ValueError:
        pass
    act_df.loc[x,'elenco'] = ' '.join(lista_x)
for i in act_df.columns:
    if i != 'elenco':
        act_df.drop(i,axis=1,inplace=True)
        
        
    
dir_df['cineastas'] = pd.Series()
for i in range(len(dir_df.columns)):
    dir_df.iloc[:,i] = dir_df.iloc[:,i].astype(str)
for x in dir_df.index:
    lista_x = list(dir_df.loc[x,:].unique())
    try:
        lista_x.remove('nan')
        lista_x.remove('None')
    except ValueError:
        pass
    dir_df.loc[x,'cineastas'] = ' '.join(lista_x)
for i in dir_df.columns:
    if i != 'cineastas':
        dir_df.drop(i,axis=1,inplace=True)
        
        
        
gen_df['genero'] = pd.Series()
for i in range(len(gen_df.columns)):
    gen_df.iloc[:,i] = gen_df.iloc[:,i].astype(str)
for x in gen_df.index:
    lista_x = list(gen_df.loc[x,:].unique())
    try:
        lista_x.remove('nan')
        lista_x.remove('None')
    except ValueError:
        pass
    gen_df.loc[x,'genero'] = ' '.join(lista_x)
for i in gen_df.columns:
    if i != 'genero':
        gen_df.drop(i,axis=1,inplace=True)

C:\Users\tinma\AppData\Local\Temp\ipykernel_25852\927220160.py:18: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  act_df['elenco'] = pd.Series()


In [245]:
act_df

,elenco
id_peli,
862,TomHanks TimAllen DonRickles JimVarney Wallace...
568,TomHanks XanderBerkeley GabrielJarret KevinBac...
13,TomHanks MykeltiWilliamson AaronMichaelLacey S...
9800,TomHanks AnnaDeavereSmith MarySteenburgen Anto...
858,TomHanks DanaIvey RossMalinger DavidHydePierce...
...,...
264269,MariyaFomina DmitriYendaltsev BorisPolunin Vik...
370213,SonitaAlidazeh
75608,RobertGardner


In [231]:
x

Index(['id_peli', 'title', 'franquicia', 'genero', 'cineastas', 'elenco'], dtype='object')

In [249]:
df = df.merge(gen_df.reset_index(),how='left').merge(dir_df.reset_index(),how='left').merge(act_df.reset_index(),how='left')
x = df.columns
df['word_bag'] = df[df.columns[1]].astype(str) + ' ' + df[df.columns[2]].astype(str).apply(lambda x: x.replace(' ','')) + ' ' + df[df.columns[3]].astype(str) + ' ' + df[df.columns[4]].astype(str) + ' ' + df[df.columns[4]].astype(str)
df['word_bag'] = df.word_bag.astype(str)
df['word_bag'] = df.word_bag.apply(lambda x: x.replace('nan',''))
for i in df.columns:
    if i not in ['word_bag','id_peli','title']:
        df.drop(i,axis=1,inplace=True)
df

,id_peli,title,word_bag
0,862,Toy Story,Toy Story ToyStoryCollection Animation Comedy ...
1,8844,Jumanji,Jumanji Family Adventure Fantasy JoeJohnston ...
2,15602,Grumpier Old Men,Grumpier Old Men GrumpyOldMenCollection Comedy...
3,31357,Waiting To Exhale,Waiting To Exhale Comedy Romance Drama Forest...
4,11862,Father Of The Bride Part Ii,Father Of The Bride Part Ii Comedy CharlesShy...
...,...,...,...
15071,432789,The Incredible Jessica James,The Incredible Jessica James Comedy Romance J...
15072,434873,It Stains The Sands Red,It Stains The Sands Red Comedy Drama Horror C...
15073,18098,Arabian Nights,Arabian Nights Family Fantasy Drama SteveBarr...
15074,455661,In A Heartbeat,In A Heartbeat Animation Comedy Family Romanc...


In [247]:
df.to_csv('clean_data/movies_model.csv',index=False)